In [1]:
import os
import random
from io import open
import unicodedata
import string
import re

import torch
# import torchaudio
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from pathlib import Path
import kaldi_io
import sys
import gc
import json
import time
from data_4 import AudioDataLoader, AudioDataset

%matplotlib inline

print_use = False

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [2]:
# train_json = "data.json"
# test_json = "data_test.json"
train_json="/home1/meichaoyang/workspace/git/Listen-Attend-Spell/egs/aishell2/dump/train/deltatrue/data.json"
test_json="/home1/meichaoyang/workspace/git/Listen-Attend-Spell/egs/aishell2/dump/test/deltatrue/data.json"
batch_size = 32
maxlen_in = 100000
maxlen_out = 30
num_workers = 8

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"]="3"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
tr_dataset = AudioDataset(train_json, batch_size,
                              maxlen_in, maxlen_out)
tr_loader = AudioDataLoader(tr_dataset, batch_size=1, num_workers=num_workers)

In [5]:
te_dataset = AudioDataset(test_json, batch_size,
                              maxlen_in, maxlen_out)
te_loader = AudioDataLoader(te_dataset, batch_size=1, num_workers=num_workers, shuffle=True)

In [6]:
char_list = []
char_list_path = "/home1/meichaoyang/workspace/git/Listen-Attend-Spell/egs/aishell2/data/lang_1char/train_chars.txt"
with open(char_list_path, "r") as f:
    for line in f:
        data = line.split()
        char_list.append(data[0])

### 模型搭建

In [13]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, vocab_size, bidirectional=True, dropout=0.0):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.rnn = nn.LSTM(input_size, hidden_size, 
                           batch_first=True,
                           dropout=dropout,
                           bidirectional=bidirectional)
        self.mlp = nn.Sequential(
            nn.Linear(self.hidden_size*2,
                      self.hidden_size),
            nn.Tanh(),
            nn.Linear(self.hidden_size, self.vocab_size))


    def forward(self, padded_input, input_lengths):
        
        total_length = padded_input.size(1)  # get the max sequence length
        packed_input = pack_padded_sequence(padded_input, input_lengths,
                                            batch_first=True)
        packed_output, hidden = self.rnn(packed_input)
        rnn_output, _ = pad_packed_sequence(packed_output,
                                        batch_first=True,
                                        total_length=total_length)
        predicted_y = self.mlp(rnn_output)
        return predicted_y

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [22]:
class CTC_Model(nn.Module):
    
    def __init__(self, input_size, hidden_size, vocab_size, bidirectional=True, dropout=0.0):
        super(CTC_Model, self).__init__()
        
        self.vocab_size = vocab_size
        self.encoder = Encoder(input_size, hidden_size, vocab_size, bidirectional, dropout)
        self.ctc_loss = nn.CTCLoss()
    
    def forward(self, padded_input, input_lengths, padded_target, target_lengths):
        
        encoder_output = self.encoder(padded_input, input_lengths)
#         print(encoder_output.shape)
        encoder_output = torch.transpose(encoder_output,0,1)
        ctc_input = encoder_output.log_softmax(2)
        
        loss = self.ctc_loss(ctc_input, padded_target, input_lengths, target_lengths)
        
        return loss

### 训练

In [23]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [24]:
def trainIters(model, epoch, optimizier, print_every=10, plot_every=10, learning_rate=0.01):

    start = time.time()
    n_iters = len(tr_loader)
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    
    criterion = nn.NLLLoss()

    for e in range(epoch):
        for i, (data) in enumerate(tr_loader):
            padded_input, input_lengths, padded_target, target_lengths= data
            padded_input = padded_input.cuda()
            input_lengths = input_lengths.cuda()
            padded_target = padded_target.cuda()
            target_lengths = target_lengths.cuda()
            loss = model(padded_input, input_lengths, padded_target, target_lengths)
    #         print(loss) #.requires_grad
            print_loss_total += float(loss)
            plot_loss_total += float(loss)

            optimizier.zero_grad()
            loss.backward()
            grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(),5)

            optimizier.step()

            if (i+1) % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                txt = 'Epoch %d | Iter %d | %s (%d %d%%) %.4f' % (e+1, i+1, timeSince(start, (e *n_iters +i+1) / (n_iters*epoch)),
                                             (i+1), (e *n_iters +i+1) / (n_iters*epoch) * 100, print_loss_avg)
                print(txt)

            if i+1 % plot_every == 0:
                plot_loss_avg = plot_loss_total / plot_every
                plot_losses.append(plot_loss_avg)
                plot_loss_total = 0

In [ ]:
input_size = 240

hidden_size = 256
vocab_size = len(char_list)
embedding_dim = 512
sos_id = 0
eos_id = 1
learning_rate = 1e-3
momentum = 0
l2 = 1e-5

IGNORE_ID=-1


model = CTC_Model(input_size, hidden_size, vocab_size)
print(model)
model.cuda()

optimizier = torch.optim.Adam(model.parameters(),
                                     lr=learning_rate,
#                                      momentum=momentum,
                                     weight_decay=l2)
trainIters(model, 20,optimizier, print_every=50)

CTC_Model(
  (encoder): Encoder(
    (rnn): LSTM(240, 256, batch_first=True, bidirectional=True)
    (mlp): Sequential(
      (0): Linear(in_features=512, out_features=256, bias=True)
      (1): Tanh()
      (2): Linear(in_features=256, out_features=5221, bias=True)
    )
  )
  (ctc_loss): CTCLoss()
)
Epoch 1 | Iter 50 | 0m 9s (- 1762m 58s) (50 0%) 107.7662
Epoch 1 | Iter 100 | 0m 15s (- 1545m 17s) (100 0%) 7.0574
Epoch 1 | Iter 150 | 0m 22s (- 1458m 53s) (150 0%) 7.0156
Epoch 1 | Iter 200 | 0m 28s (- 1397m 50s) (200 0%) 6.9890
Epoch 1 | Iter 250 | 0m 34s (- 1357m 22s) (250 0%) 6.9900
Epoch 1 | Iter 300 | 0m 40s (- 1323m 23s) (300 0%) 6.9735
Epoch 1 | Iter 350 | 0m 46s (- 1296m 23s) (350 0%) 6.9726
Epoch 1 | Iter 400 | 0m 52s (- 1278m 1s) (400 0%) 6.9688
Epoch 1 | Iter 450 | 0m 58s (- 1259m 43s) (450 0%) 6.9342
Epoch 1 | Iter 500 | 1m 3s (- 1241m 26s) (500 0%) 6.9600
Epoch 1 | Iter 550 | 1m 9s (- 1225m 41s) (550 0%) 6.9193
Epoch 1 | Iter 600 | 1m 14s (- 1211m 29s) (600 0%) 6.9634
Epoch

In [11]:
input_size = 240

hidden_size = 256
vocab_size = len(char_list)
embedding_dim = 512
sos_id = 0
eos_id = 1
learning_rate = 1e-3
momentum = 0
l2 = 1e-5

IGNORE_ID=-1


model = CTC_Model(input_size, hidden_size)
print(model)
model.cuda()

optimizier = torch.optim.Adam(model.parameters(),
                                     lr=learning_rate,
#                                      momentum=momentum,
                                     weight_decay=l2)
trainIters(model, 20,optimizier, print_every=1)

CTC_Model(
  (encoder): Encoder(
    (rnn): LSTM(240, 256, batch_first=True, bidirectional=True)
  )
  (ctc_loss): CTCLoss()
)
CTC_Model(
  (encoder): Encoder(
    (rnn): LSTM(240, 256, batch_first=True, bidirectional=True)
  )
  (ctc_loss): CTCLoss()
)
Epoch 1 | Iter 1 | 0m 1s (- 11968m 10s) (1 0%) 399.1771
Epoch 1 | Iter 1 | 0m 1s (- 11968m 10s) (1 0%) 399.1771
Epoch 1 | Iter 2 | 0m 1s (- 7345m 41s) (2 0%) 400.1349
Epoch 1 | Iter 3 | 0m 1s (- 5421m 59s) (3 0%) 345.4847
Epoch 1 | Iter 2 | 0m 1s (- 7345m 41s) (2 0%) 400.1349
Epoch 1 | Iter 3 | 0m 1s (- 5421m 59s) (3 0%) 345.4847
Epoch 1 | Iter 4 | 0m 1s (- 4433m 53s) (4 0%) 375.8539
Epoch 1 | Iter 5 | 0m 1s (- 3841m 57s) (5 0%) 309.4611
Epoch 1 | Iter 4 | 0m 1s (- 4433m 53s) (4 0%) 375.8539
Epoch 1 | Iter 5 | 0m 1s (- 3841m 57s) (5 0%) 309.4611
Epoch 1 | Iter 6 | 0m 2s (- 3429m 55s) (6 0%) 303.2388
Epoch 1 | Iter 7 | 0m 2s (- 3135m 4s) (7 0%) 298.9521
Epoch 1 | Iter 6 | 0m 2s (- 3429m 55s) (6 0%) 303.2388
Epoch 1 | Iter 7 | 0m 2s (- 31

Epoch 1 | Iter 72 | 0m 10s (- 1376m 5s) (72 0%) 200.0567
Epoch 1 | Iter 73 | 0m 10s (- 1370m 58s) (73 0%) 187.4937
Epoch 1 | Iter 72 | 0m 10s (- 1376m 5s) (72 0%) 200.0567
Epoch 1 | Iter 73 | 0m 10s (- 1370m 58s) (73 0%) 187.4937
Epoch 1 | Iter 74 | 0m 10s (- 1367m 2s) (74 0%) 191.9516
Epoch 1 | Iter 75 | 0m 10s (- 1363m 26s) (75 0%) 208.6974
Epoch 1 | Iter 74 | 0m 10s (- 1367m 2s) (74 0%) 191.9516
Epoch 1 | Iter 75 | 0m 10s (- 1363m 26s) (75 0%) 208.6974
Epoch 1 | Iter 76 | 0m 10s (- 1359m 39s) (76 0%) 197.0114
Epoch 1 | Iter 77 | 0m 10s (- 1358m 56s) (77 0%) 189.8945
Epoch 1 | Iter 76 | 0m 10s (- 1359m 39s) (76 0%) 197.0114
Epoch 1 | Iter 77 | 0m 10s (- 1358m 56s) (77 0%) 189.8945
Epoch 1 | Iter 78 | 0m 10s (- 1355m 40s) (78 0%) 191.3541
Epoch 1 | Iter 79 | 0m 10s (- 1352m 49s) (79 0%) 189.2785
Epoch 1 | Iter 78 | 0m 10s (- 1355m 40s) (78 0%) 191.3541
Epoch 1 | Iter 79 | 0m 10s (- 1352m 49s) (79 0%) 189.2785
Epoch 1 | Iter 80 | 0m 11s (- 1349m 0s) (80 0%) 190.8376
Epoch 1 | Iter 81 |

Epoch 1 | Iter 142 | 0m 18s (- 1251m 19s) (142 0%) 183.6001
Epoch 1 | Iter 143 | 0m 18s (- 1249m 56s) (143 0%) 184.5017
Epoch 1 | Iter 142 | 0m 18s (- 1251m 19s) (142 0%) 183.6001
Epoch 1 | Iter 143 | 0m 18s (- 1249m 56s) (143 0%) 184.5017
Epoch 1 | Iter 144 | 0m 18s (- 1249m 1s) (144 0%) 183.4916
Epoch 1 | Iter 145 | 0m 18s (- 1247m 32s) (145 0%) 176.2198
Epoch 1 | Iter 144 | 0m 18s (- 1249m 1s) (144 0%) 183.4916
Epoch 1 | Iter 145 | 0m 18s (- 1247m 32s) (145 0%) 176.2198
Epoch 1 | Iter 146 | 0m 18s (- 1246m 25s) (146 0%) 177.9950
Epoch 1 | Iter 147 | 0m 18s (- 1244m 52s) (147 0%) 181.6489
Epoch 1 | Iter 146 | 0m 18s (- 1246m 25s) (146 0%) 177.9950
Epoch 1 | Iter 147 | 0m 18s (- 1244m 52s) (147 0%) 181.6489
Epoch 1 | Iter 148 | 0m 18s (- 1243m 33s) (148 0%) 175.7631
Epoch 1 | Iter 149 | 0m 19s (- 1242m 18s) (149 0%) 174.9800
Epoch 1 | Iter 148 | 0m 18s (- 1243m 33s) (148 0%) 175.7631
Epoch 1 | Iter 149 | 0m 19s (- 1242m 18s) (149 0%) 174.9800
Epoch 1 | Iter 150 | 0m 19s (- 1241m 17s) 

Epoch 1 | Iter 210 | 0m 26s (- 1204m 35s) (210 0%) 173.0214
Epoch 1 | Iter 211 | 0m 26s (- 1204m 25s) (211 0%) 173.7970
Epoch 1 | Iter 212 | 0m 26s (- 1204m 0s) (212 0%) 175.4187
Epoch 1 | Iter 213 | 0m 26s (- 1203m 9s) (213 0%) 179.7552
Epoch 1 | Iter 212 | 0m 26s (- 1204m 0s) (212 0%) 175.4187
Epoch 1 | Iter 213 | 0m 26s (- 1203m 9s) (213 0%) 179.7552
Epoch 1 | Iter 214 | 0m 26s (- 1202m 28s) (214 0%) 170.4618
Epoch 1 | Iter 215 | 0m 26s (- 1201m 35s) (215 0%) 181.7186
Epoch 1 | Iter 214 | 0m 26s (- 1202m 28s) (214 0%) 170.4618
Epoch 1 | Iter 215 | 0m 26s (- 1201m 35s) (215 0%) 181.7186
Epoch 1 | Iter 216 | 0m 26s (- 1200m 44s) (216 0%) 176.8787
Epoch 1 | Iter 217 | 0m 26s (- 1200m 2s) (217 0%) 175.0920
Epoch 1 | Iter 216 | 0m 26s (- 1200m 44s) (216 0%) 176.8787
Epoch 1 | Iter 217 | 0m 26s (- 1200m 2s) (217 0%) 175.0920
Epoch 1 | Iter 218 | 0m 26s (- 1199m 14s) (218 0%) 170.1087
Epoch 1 | Iter 219 | 0m 26s (- 1198m 20s) (219 0%) 171.2564
Epoch 1 | Iter 218 | 0m 26s (- 1199m 14s) (218

Epoch 1 | Iter 278 | 0m 33s (- 1169m 8s) (278 0%) 201.6042
Epoch 1 | Iter 279 | 0m 33s (- 1168m 31s) (279 0%) 165.8486
Epoch 1 | Iter 280 | 0m 33s (- 1167m 50s) (280 0%) 172.0959
Epoch 1 | Iter 281 | 0m 33s (- 1167m 9s) (281 0%) 168.7124
Epoch 1 | Iter 282 | 0m 33s (- 1166m 34s) (282 0%) 172.6881
Epoch 1 | Iter 281 | 0m 33s (- 1167m 9s) (281 0%) 168.7124
Epoch 1 | Iter 282 | 0m 33s (- 1166m 34s) (282 0%) 172.6881
Epoch 1 | Iter 283 | 0m 33s (- 1165m 59s) (283 0%) 163.2892
Epoch 1 | Iter 284 | 0m 34s (- 1165m 16s) (284 0%) 171.4239
Epoch 1 | Iter 285 | 0m 34s (- 1164m 37s) (285 0%) 164.1865
Epoch 1 | Iter 283 | 0m 33s (- 1165m 59s) (283 0%) 163.2892
Epoch 1 | Iter 284 | 0m 34s (- 1165m 16s) (284 0%) 171.4239
Epoch 1 | Iter 285 | 0m 34s (- 1164m 37s) (285 0%) 164.1865
Epoch 1 | Iter 286 | 0m 34s (- 1163m 57s) (286 0%) 166.6825
Epoch 1 | Iter 287 | 0m 34s (- 1163m 31s) (287 0%) 175.0222
Epoch 1 | Iter 286 | 0m 34s (- 1163m 57s) (286 0%) 166.6825
Epoch 1 | Iter 287 | 0m 34s (- 1163m 31s) (

...

Epoch 1 | Iter 6560 | 8m 46s (- 772m 15s) (6560 1%) 129.1609
Epoch 1 | Iter 6561 | 8m 46s (- 772m 13s) (6561 1%) 130.1179
Epoch 1 | Iter 6562 | 8m 46s (- 772m 12s) (6562 1%) 132.0822
Epoch 1 | Iter 6563 | 8m 46s (- 772m 10s) (6563 1%) 138.6285
Epoch 1 | Iter 6564 | 8m 46s (- 772m 9s) (6564 1%) 136.1473
Epoch 1 | Iter 6565 | 8m 46s (- 772m 8s) (6565 1%) 123.3998
Epoch 1 | Iter 6566 | 8m 47s (- 772m 7s) (6566 1%) 122.1905
Epoch 1 | Iter 6567 | 8m 47s (- 772m 5s) (6567 1%) 129.1123
Epoch 1 | Iter 6564 | 8m 46s (- 772m 9s) (6564 1%) 136.1473
Epoch 1 | Iter 6565 | 8m 46s (- 772m 8s) (6565 1%) 123.3998
Epoch 1 | Iter 6566 | 8m 47s (- 772m 7s) (6566 1%) 122.1905
Epoch 1 | Iter 6567 | 8m 47s (- 772m 5s) (6567 1%) 129.1123
Epoch 1 | Iter 6568 | 8m 47s (- 772m 3s) (6568 1%) 135.4609
Epoch 1 | Iter 6569 | 8m 47s (- 772m 3s) (6569 1%) 140.8806
Epoch 1 | Iter 6570 | 8m 47s (- 772m 1s) (6570 1%) 131.9287
Epoch 1 | Iter 6571 | 8m 47s (- 772m 0s) (6571 1%) 123.3071
Epoch 1 | Iter 6568 | 8m 47s (- 772m

Epoch 1 | Iter 6629 | 8m 51s (- 771m 0s) (6629 1%) 136.7830
Epoch 1 | Iter 6630 | 8m 51s (- 770m 58s) (6630 1%) 134.0070
Epoch 1 | Iter 6631 | 8m 51s (- 770m 57s) (6631 1%) 132.7003
Epoch 1 | Iter 6632 | 8m 51s (- 770m 56s) (6632 1%) 135.0272
Epoch 1 | Iter 6633 | 8m 51s (- 770m 55s) (6633 1%) 118.1433
Epoch 1 | Iter 6634 | 8m 51s (- 770m 53s) (6634 1%) 122.7223
Epoch 1 | Iter 6635 | 8m 51s (- 770m 52s) (6635 1%) 131.5896
Epoch 1 | Iter 6633 | 8m 51s (- 770m 55s) (6633 1%) 118.1433
Epoch 1 | Iter 6634 | 8m 51s (- 770m 53s) (6634 1%) 122.7223
Epoch 1 | Iter 6635 | 8m 51s (- 770m 52s) (6635 1%) 131.5896
Epoch 1 | Iter 6636 | 8m 51s (- 770m 51s) (6636 1%) 121.1726
Epoch 1 | Iter 6637 | 8m 51s (- 770m 50s) (6637 1%) 132.3755
Epoch 1 | Iter 6638 | 8m 51s (- 770m 49s) (6638 1%) 133.2229
Epoch 1 | Iter 6636 | 8m 51s (- 770m 51s) (6636 1%) 121.1726
Epoch 1 | Iter 6637 | 8m 51s (- 770m 50s) (6637 1%) 132.3755
Epoch 1 | Iter 6638 | 8m 51s (- 770m 49s) (6638 1%) 133.2229
Epoch 1 | Iter 6639 | 8m 

Epoch 1 | Iter 6697 | 8m 55s (- 769m 33s) (6697 1%) 137.2844
Epoch 1 | Iter 6698 | 8m 55s (- 769m 32s) (6698 1%) 136.1261
Epoch 1 | Iter 6699 | 8m 56s (- 769m 30s) (6699 1%) 140.0025
Epoch 1 | Iter 6700 | 8m 56s (- 769m 28s) (6700 1%) 131.0531
Epoch 1 | Iter 6701 | 8m 56s (- 769m 27s) (6701 1%) 132.3134
Epoch 1 | Iter 6702 | 8m 56s (- 769m 24s) (6702 1%) 129.4655
Epoch 1 | Iter 6703 | 8m 56s (- 769m 23s) (6703 1%) 133.0455
Epoch 1 | Iter 6704 | 8m 56s (- 769m 21s) (6704 1%) 124.6525
Epoch 1 | Iter 6701 | 8m 56s (- 769m 27s) (6701 1%) 132.3134
Epoch 1 | Iter 6702 | 8m 56s (- 769m 24s) (6702 1%) 129.4655
Epoch 1 | Iter 6703 | 8m 56s (- 769m 23s) (6703 1%) 133.0455
Epoch 1 | Iter 6704 | 8m 56s (- 769m 21s) (6704 1%) 124.6525
Epoch 1 | Iter 6705 | 8m 56s (- 769m 20s) (6705 1%) 134.9238
Epoch 1 | Iter 6706 | 8m 56s (- 769m 18s) (6706 1%) 131.5156
Epoch 1 | Iter 6705 | 8m 56s (- 769m 20s) (6705 1%) 134.9238
Epoch 1 | Iter 6706 | 8m 56s (- 769m 18s) (6706 1%) 131.5156


RuntimeError: cuDNN error: CUDNN_STATUS_EXECUTION_FAILED
Exception raised from _cudnn_rnn_backward_input at /pytorch/aten/src/ATen/native/cudnn/RNN.cpp:923 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::string) + 0x42 (0x7ff9dcdd61e2 in /home1/meichaoyang/anaconda3/lib/python3.8/site-packages/torch/lib/libc10.so)
frame #1: <unknown function> + 0xee132f (0x7ff9de12032f in /home1/meichaoyang/anaconda3/lib/python3.8/site-packages/torch/lib/libtorch_cuda.so)
frame #2: at::native::_cudnn_rnn_backward(at::Tensor const&, c10::ArrayRef<at::Tensor>, long, at::Tensor const&, at::Tensor const&, at::Tensor const&, at::Tensor const&, at::Tensor const&, at::Tensor const&, at::Tensor const&, long, long, long, bool, double, bool, bool, c10::ArrayRef<long>, at::Tensor const&, at::Tensor const&, std::array<bool, 4ul>) + 0x1a9 (0x7ff9de1247c9 in /home1/meichaoyang/anaconda3/lib/python3.8/site-packages/torch/lib/libtorch_cuda.so)
frame #3: <unknown function> + 0xf1dbf7 (0x

RuntimeError: cuDNN error: CUDNN_STATUS_EXECUTION_FAILED
Exception raised from _cudnn_rnn_backward_input at /pytorch/aten/src/ATen/native/cudnn/RNN.cpp:923 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::string) + 0x42 (0x7ff9dcdd61e2 in /home1/meichaoyang/anaconda3/lib/python3.8/site-packages/torch/lib/libc10.so)
frame #1: <unknown function> + 0xee132f (0x7ff9de12032f in /home1/meichaoyang/anaconda3/lib/python3.8/site-packages/torch/lib/libtorch_cuda.so)
frame #2: at::native::_cudnn_rnn_backward(at::Tensor const&, c10::ArrayRef<at::Tensor>, long, at::Tensor const&, at::Tensor const&, at::Tensor const&, at::Tensor const&, at::Tensor const&, at::Tensor const&, at::Tensor const&, long, long, long, bool, double, bool, bool, c10::ArrayRef<long>, at::Tensor const&, at::Tensor const&, std::array<bool, 4ul>) + 0x1a9 (0x7ff9de1247c9 in /home1/meichaoyang/anaconda3/lib/python3.8/site-packages/torch/lib/libtorch_cuda.so)
frame #3: <unknown function> + 0xf1dbf7 (0x

In [ ]:
for i, (data) in enumerate(tr_loader):